### DRS ids on Seven Bridges platform
This notebook explores the various DRS Ids acquired by the same file as it is imported into different instances of the Seven Bridges platform. In this case the NCI Cancer Genomics Cloud (CGC) and BioDataCatalyst are used.

These files are loaded to the CNest project.

In [8]:
import json

from fasp.loc import sbcgcDRSClient
drsClient = sbcgcDRSClient('/Users/forei/.keys/sbcgc_key.json', 's3')
drs_id ='62765a94f26c9351737fa8cc'
print(json.dumps(drsClient.get_object(drs_id),indent=3))

{
   "id": "62765a94f26c9351737fa8cc",
   "name": "HG00445.test.cram",
   "size": 3971394,
   "checksums": [
      {
         "type": "etag",
         "checksum": "6d5a0c75eae2b7ff704e61a8d07b931e-1"
      }
   ],
   "self_uri": "drs://cgc-ga4gh-api.sbgenomics.com/62765a94f26c9351737fa8cc",
   "created_time": "2022-05-07T11:40:04Z",
   "updated_time": "2022-05-07T11:40:04Z",
   "mime_type": "application/json",
   "access_methods": [
      {
         "type": "s3",
         "region": "us-east-1",
         "access_id": "aws-us-east-1"
      }
   ]
}


This the same file imported to the CGC via the DRS import capability. Note that it has acquired a different DRS id. That DRS id is used in computes initiated on the file.

In [20]:
sb_imported_drs_drs_id ='626c1775f26c93517368c97c'
resp = drsClient.get_object(sb_imported_drs_drs_id)
print(json.dumps(drsClient.get_object(sb_imported_drs_drs_id),indent=3))
#print(resp.raise_for_status())

{
   "id": "626c1775f26c93517368c97c",
   "name": "HG00445.final.cram",
   "size": 18276728210,
   "checksums": [
      {
         "type": "etag",
         "checksum": "a06d149487e09844cca481517c9e8108-2179"
      }
   ],
   "self_uri": "drs://cgc-ga4gh-api.sbgenomics.com/626c1775f26c93517368c97c",
   "created_time": "2022-04-29T16:51:01Z",
   "updated_time": "2022-04-29T16:51:01Z",
   "mime_type": "application/json",
   "access_methods": [
      {
         "type": "s3",
         "region": "us-east-1",
         "access_id": "aws-us-east-1"
      }
   ]
}


Show the URL obtained to access the file

In [25]:
indirect_url = drsClient.get_access_url(sb_imported_drs_drs_id)
#print the url without any of the token information
print(indirect_url.split('?')[0])

https://nih-nhlbi-biodata-catalyst-1000-genomes-high-coverage.s3.amazonaws.com/CCDG_13607/Project_CCDG_13607_B01_GRM_WGS.cram.2019-02-06/Sample_HG00445/analysis/HG00445.final.cram


### The same file on the source platform (BioDataCatalyst)
What does this file look like on the original platform? TopMed
The Biodata Catalyst DRS id for the 'imported' file is 626c079e645ccb7324c671d1

We can access it directly from BioDataCatalyst as follows

In [26]:
from fasp.loc import sbbdcDRSClient
bdc_drs_id ='626c079e645ccb7324c671d1'
bdc_drsClient = sbbdcDRSClient('/Users/forei/.keys/sbbdc_key.json', 's3')
bdc_drsClient.get_object(bdc_drs_id)

{'id': '626c079e645ccb7324c671d1',
 'name': 'HG00445.final.cram',
 'size': 18276728210,
 'checksums': [{'type': 'etag',
   'checksum': 'a06d149487e09844cca481517c9e8108-2179'}],
 'self_uri': 'drs://ga4gh-api.sb.biodatacatalyst.nhlbi.nih.gov/626c079e645ccb7324c671d1',
 'created_time': '2022-04-29T15:43:26Z',
 'updated_time': '2022-10-17T11:50:03Z',
 'mime_type': 'application/json',
 'access_methods': [{'type': 's3',
   'region': 'us-east-1',
   'access_id': 'aws-us-east-1'}]}

In [27]:
direct_url = bdc_drsClient.get_access_url(bdc_drs_id)
#print the url without any of the token information
print("URL provided via BioDataCatalyst DRS")
print(direct_url.split('?')[0])
print("URL provided via CGC DRS")
print(indirect_url.split('?')[0])

URL provided via BioDataCatalyst DRS
https://nih-nhlbi-biodata-catalyst-1000-genomes-high-coverage.s3.amazonaws.com/CCDG_13607/Project_CCDG_13607_B01_GRM_WGS.cram.2019-02-06/Sample_HG00445/analysis/HG00445.final.cram
URL provided via CGC DRS
https://nih-nhlbi-biodata-catalyst-1000-genomes-high-coverage.s3.amazonaws.com/CCDG_13607/Project_CCDG_13607_B01_GRM_WGS.cram.2019-02-06/Sample_HG00445/analysis/HG00445.final.cram


Can we run the compute off CGC using WES and passing the BDC DRS id?
Worth checking.

This was attempted but CGC could not used the BDC DRS id